[View in Colaboratory](https://colab.research.google.com/github/suhan-s/google-collab/blob/master/cnn_mnist_tensorboard.ipynb)

# **Setting Up Ngrok**

In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2018-10-19 14:12:53--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.71.139.107, 52.45.84.34, 52.45.111.123, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.71.139.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  --.-KB/s    in 0.1s    

2018-10-19 14:12:54 (39.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://97392c83.ngrok.io


# **Cnn Mnist With Tensorboard Integration**

In [2]:
import tensorflow as tf
import os
import os.path

tf.logging.set_verbosity(tf.logging.ERROR)

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
tf.reset_default_graph() 
x = tf.placeholder(tf.float32, shape=[None, 784], name='Initial_Image')
y_ = tf.placeholder(tf.float32, shape=[None, 10], name='Output')
x_image = tf.reshape(x, [-1, 28, 28, 1], name='Transformed_Image')
tf.summary.image('input', x_image, 3)
keep_prob = tf.placeholder(tf.float32, name='Dropout_Probability')

with tf.name_scope("conv1"):
    conv1 = tf.layers.conv2d(x_image, 32, 5, 1, padding='SAME', activation=tf.nn.relu, name="conv1")
    maxpool1 = tf.layers.max_pooling2d(conv1, 2, 2, name="maxpool1")

with tf.name_scope("conv2"):
    conv2 = tf.layers.conv2d(maxpool1, 64, 5, 1, padding='SAME', activation=tf.nn.relu, name="conv2")
    maxpool2 = tf.layers.max_pooling2d(conv2, 2, 2, name="maxpool2")

with tf.name_scope("fc"):
    fc1 = tf.layers.flatten(maxpool2)
    fc2 = tf.layers.dense(fc1, 1024)
    fc3 = tf.layers.dropout(fc2, keep_prob)

with tf.name_scope("out"):
    out = tf.layers.dense(fc3, 10)

with tf.name_scope("enthropy"):
    cross_enthropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=out))
    tf.summary.scalar("enthropy", cross_enthropy)

with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cross_enthropy)

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.arg_max(out, 1), tf.arg_max(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
    tf.summary.scalar("accuracy", accuracy)

summ = tf.summary.merge_all()

session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())

saver = tf.train.Saver()

writer = tf.summary.FileWriter(LOG_DIR)
writer.add_graph(session.graph)

for steps in range(2000):
    batch = mnist.train.next_batch(100)
    if steps % 5 == 0:
        [train_accuracy, s] = session.run([accuracy, summ], feed_dict={x: batch[0], y_: batch[1], keep_prob: 1})
        writer.add_summary(s, steps)
    if steps % 500 == 0:
        print("Step %d accuracy %g" % (steps, train_accuracy))
        saver.save(session, os.path.join(LOG_DIR, "model.ckpt"), steps)  
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("Done")
print("Traing Accuracy %g"% accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1}))
session.close()


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Step 0 accuracy 0.07
Step 500 accuracy 0.97
Step 1000 accuracy 0.95
Step 1500 accuracy 0.98
Done
Traing Accuracy 0.9857
